In [4]:
# Author: Han Liu
# Date: 09/20/2020

import numpy as np
import math
import sys
from qiskit.tools.visualization import plot_histogram, plot_state_city
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister


# The target distribution
target_distr = np.array([0, 0.5, 0.5, 0])

# Initialize the number of layers and the number of parameters. 
num_layers=5
num_params=num_layers*2


##############Error model##########################
from qiskit.providers.aer.noise import NoiseModel
from qiskit.providers.aer.noise.errors import pauli_error, depolarizing_error

#Probabilities for the error model
p_m=0.01
p_g=0.01

def get_noise(p_meas,p_gate):

    error_meas = pauli_error([('X',p_meas), ('I', 1 - p_meas)])
    error_gate1 = depolarizing_error(p_gate, 1)
    error_gate2 = error_gate1.tensor(error_gate1)

    noise_model = NoiseModel()
    noise_model.add_all_qubit_quantum_error(error_meas, "measure") # measurement error is applied to measurements
    noise_model.add_all_qubit_quantum_error(error_gate1, ["rx"]) # single qubit gate error is applied to rx gates
    noise_model.add_all_qubit_quantum_error(error_gate1, ["ry"]) # single qubit gate error is applied to ry gates
    noise_model.add_all_qubit_quantum_error(error_gate2, ["cx"]) # two qubit gate error is applied to cx gates
        
    return noise_model
noise_model = get_noise(p_m,p_g)
###################################################






##################Ansatz circuit####################### 
def get_var_forma(params):
    if((num_layers < 1) or (len(params)!=num_params)): 
        print("Error!") 
        sys.exit(0)
    else:        
        circ = QuantumCircuit(2, 2)              
        k=0
        for iz in range (0, 2):
            circ.ry(params[k], iz)
            k=k+1
            
    i = 1        
    while i < num_layers:        
        if(i%2 == 0): 
            circ.cx(0, 1)
            for iz in range (0, 2):
                circ.ry(params[k], iz)
                k=k+1
        else:
            circ.cx(1, 0)
            for iz in range (0, 2):
                circ.rx(params[k], iz)
                k=k+1                
        i=i+1         
        
    circ.measure([0,1], [0,1])    
    return circ


def get_var_form(params):
    if((num_layers < 1) or (len(params)!=num_params)): 
        print("Error!") 
        sys.exit(0)
    else:        
        circ = QuantumCircuit(2, 2)              
        k=0
        for iz in range (0, 2):
            circ.ry(params[k], iz)
            k=k+1
            
    i = 1        
    while i < num_layers:        
        if(i%2 == 0): 
            circ.cx(0, 1)
            for iz in range (0, 2):
                circ.ry(params[k], iz)
                k=k+1
        else:
            circ.cx(0, 1)
            for iz in range (0, 2):
                circ.ry(params[k], iz)
                k=k+1                
        i=i+1         
        
    circ.measure([0,1], [0,1])    
    return circ
#######################################################
    
                

        
        
###############Get probability distribution ###########
def get_probability_distribution(counts):
    output_distr = [v / NUM_SHOTS for v in counts.values()]
    if len(output_distr) == 1:
        output_distr.append(0)
        output_distr.append(0)
        output_distr.append(0)
    if len(output_distr) == 2:
        output_distr.append(0)
        output_distr.append(0)
    if len(output_distr) == 3:
        output_distr.append(0)                        
    return output_distr
#######################################################




##################Cost function########################
def objective_function(params):
    # Obtain a quantum circuit instance from the paramters
    circ = get_var_form(params)
    # Execute the quantum circuit to obtain the probability distribution associated with the current parameters, 
    # including the noise model
    result = execute(circ, backend, shots=NUM_SHOTS, noise_model=noise_model).result()
    # Obtain the counts for each measured state, and convert those counts into a probability vector
    output_distr = get_probability_distribution(result.get_counts(circ))
    # Calculate the cost as the distance between the output distribution and the target distribution
    cost = sum([np.abs(output_distr[i] - target_distr[i]) for i in range(4)])
    return cost
######################################################



########################Main########################
#from qiskit import Aer, execute
from qiskit import Aer, execute

# Initialize the optimizer
from qiskit.aqua.components.optimizers import COBYLA, SPSA
backend = Aer.get_backend("qasm_simulator")
#optimizer = SPSA(save_steps=1, last_avg=1, c0=1.6, c1=0.1, c2=0.602, c3=0.101, c4=0, skip_calibration=False, max_trials=100)
optimizer = COBYLA(maxiter=500, tol=0.0001)

#set num of measurements/iteration
for i in range(0,5):

    NUM_SHOTS = 10**(i)

    # Create the initial parameters
    params = np.random.rand(num_params)

    # Optimization
    op = optimizer.optimize(num_vars=num_params, objective_function=objective_function, initial_point=params)

    # Obtain the output distribution using the final parameters
    circ = get_var_form(op[0])
    counts = execute(circ, backend, shots=NUM_SHOTS, noise_model=noise_model).result().get_counts(circ)
    output_distr = get_probability_distribution(counts)


    # Output
    #plot_histogram(counts, title='num of measurements = %d'%(NUM_SHOTS))
    print('num of measurements = %d'%(NUM_SHOTS))
    print("Target Distribution:", target_distr)
    print("Obtained Distribution:", output_distr)
    print("Output Error (Manhattan Distance):", op[1])
    print("Parameters Found:", op[0])        
###################################################



num of measurements = 1
Target Distribution: [0.  0.5 0.5 0. ]
Obtained Distribution: [1.0, 0, 0, 0]
Output Error (Manhattan Distance): 2.0
Parameters Found: [0.11478068 0.27061956 0.15428952 0.73507863 0.09023688 0.13394963
 0.90304685 0.13303202 0.2852655  0.01755462]
num of measurements = 10
Target Distribution: [0.  0.5 0.5 0. ]
Obtained Distribution: [0.1, 0.2, 0.7, 0]
Output Error (Manhattan Distance): 1.0
Parameters Found: [ 2.12697722  0.97098718  1.99782372  1.89196288  0.61188064 -0.07443599
  0.48582066 -0.17614221 -0.25562181  0.71625751]
num of measurements = 100
Target Distribution: [0.  0.5 0.5 0. ]
Obtained Distribution: [0.03, 0.48, 0.05, 0.44]
Output Error (Manhattan Distance): 0.8600000000000001
Parameters Found: [0.07558544 0.99153261 1.68622663 0.09117945 0.13515855 0.10838759
 0.54007395 0.87403481 0.27661647 0.49304604]
num of measurements = 1000
Target Distribution: [0.  0.5 0.5 0. ]
Obtained Distribution: [0.012, 0.449, 0.512, 0.027]
Output Error (Manhattan Dis